# Сохранение csv (train, test)

In [42]:
import pandas as pd

raw_crowd_test = pd.read_csv("C://Users//a.karakuchukova//Downloads//crowd//raw_crowd_test.tsv", sep="\t")
raw_crowd_train = pd.read_csv("C://Users//a.karakuchukova//Downloads//crowd//train//raw_crowd_train.tsv", sep="\t")

In [48]:
raw_crowd_test.shape, raw_crowd_train.shape

((78205, 9), (898136, 9))

In [45]:
raw_crowd_train.columns

Index(['hash_id', 'audio_path', 'duration', 'annotator_emo', 'golden_emo',
       'annotator_id', 'speaker_text', 'speaker_emo', 'source_id'],
      dtype='object')

In [46]:
raw_crowd_test.head(1)

,hash_id,audio_path,duration,annotator_emo,golden_emo,annotator_id,speaker_text,speaker_emo,source_id
0,9e9961c53ca6eeb440b217e539fbf46c,wavs/9e9961c53ca6eeb440b217e539fbf46c.wav,5.82,neutral,NaN,858305a5450b7bd1288ba0053b1cd1c1,я слушаю,neutral,4282ddc30d71ef420e202e0c60391e9f


In [47]:
raw_crowd_train.head(1)

,hash_id,audio_path,duration,annotator_emo,golden_emo,annotator_id,speaker_text,speaker_emo,source_id
0,475e76f77ac1ed7cabafca740b15b32a,wavs/475e76f77ac1ed7cabafca740b15b32a.wav,2.453,angry,NaN,858305a5450b7bd1288ba0053b1cd1c1,не надо не надо не надо не надо,angry,fa136da095807ea6cd18dd6e2f58d4d0


In [44]:
len(raw_crowd_test.speaker_text.unique())

16372

In [43]:
len(raw_crowd_train.speaker_text.unique())

122960

In [50]:
crowd_train = raw_crowd_train[['annotator_emo', 'speaker_text', 'speaker_emo']].drop_duplicates().dropna()
crowd_test = raw_crowd_test[['annotator_emo', 'speaker_text', 'speaker_emo']].drop_duplicates().dropna()

In [55]:
sorted(crowd_train.annotator_emo.unique()), sorted(crowd_test.annotator_emo.unique())

(['angry', 'neutral', 'other', 'positive', 'sad'],
 ['angry', 'neutral', 'other', 'positive', 'sad'])

In [56]:
len(crowd_train.speaker_text.unique()), len(crowd_test.speaker_text.unique())

(122959, 16371)

In [ ]:
crowd_train.to_csv('crowd_train.csv')
crowd_test.to_csv('crowd_test.csv')

# Проблема: некоторым speaker_text соотвествуют несколько эмоций

In [57]:
crowd_train.groupby('speaker_text').agg({'annotator_emo':'count'}).reset_index()[crowd_train.groupby('speaker_text').agg({'annotator_emo':'count'}).reset_index().annotator_emo > 1]

,speaker_text,annotator_emo
2,2gis брюс ли,2
5,2gis тобой все в порядке,2
7,a a a a a a good man man,2
11,a r rahman dream zone fire,2
13,a давай веселей мне рассказал шуточки,2
...,...,...
122951,яхта парус,3
122953,яхты обзор,2
122955,яцко три аккорда,2
122956,ячейки из под яиц,3


In [58]:
crowd_train[crowd_train.speaker_text == '2gis брюс ли']

,annotator_emo,speaker_text,speaker_emo
275374,neutral,2gis брюс ли,angry
276782,positive,2gis брюс ли,angry


In [59]:
duplicates = crowd_train.groupby("speaker_text")["annotator_emo"].nunique()
conflicting_texts = duplicates[duplicates > 1].index
conflict_set = set(conflicting_texts)
df_clean = crowd_train[~crowd_train["speaker_text"].isin(conflict_set)]

In [61]:
df_clean.annotator_emo.value_counts()

annotator_emo
neutral     45694
angry        5437
positive     4682
sad          3672
other         422
Name: count, dtype: int64

In [62]:
crowd_train.annotator_emo.value_counts()

annotator_emo
neutral     134695
sad          49607
positive     27967
angry        20348
other         4352
Name: count, dtype: int64

In [ ]:
label_distribution = (
    crowd_train.groupby("speaker_text")["annotator_emo"]
    .value_counts(normalize=True)
    .rename("confidence")
    .reset_index()
)

label_agg = label_distribution.groupby("speaker_text").agg({
    "annotator_emo": "first",  
    "confidence": "max"      
}).reset_index()

label_agg.columns = ["speaker_text", "final_emotion", "label_confidence"]
label_agg.head(2)

,speaker_text,final_emotion,label_confidence
0,2drots литвин,neutral,1.0
1,2gis,neutral,1.0


In [94]:
label_agg.label_confidence.unique()

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
       0.66666667, 0.4       , 0.6       , 0.42857143, 0.75      ,
       0.28571429, 0.375     ])

In [95]:
label_agg.shape

(122959, 3)

In [ ]:
df_train = label_agg
print(df_train.shape)
df_train = df_train[df_train["speaker_text"].notna()]
print(df_train.shape)
df_train = df_train[df_train["speaker_text"].str.len() >= 5]
print(df_train.shape)

(122959, 3)
(122959, 3)
(122257, 3)


In [ ]:
df_train = df_train[df_train["label_confidence"] >= 0.5]
print(df_train.shape)

(114153, 3)


In [98]:
df_train["final_emotion"].value_counts(normalize=True)

final_emotion
neutral     0.667122
sad         0.139366
positive    0.102547
angry       0.081802
other       0.009163
Name: proportion, dtype: float64

In [ ]:
df_train["final_emotion"] = df_train["final_emotion"].replace("other", "neutral")

df_train["final_emotion"].value_counts(normalize=True)

final_emotion
neutral     0.676285
sad         0.139366
positive    0.102547
angry       0.081802
Name: proportion, dtype: float64

In [100]:
df_train.drop(columns = 'label_confidence', inplace = True)

In [101]:
df_train.to_csv('dusha_crowd_train_processed.csv')

In [ ]:
label_distribution = (
    crowd_test.groupby("speaker_text")["annotator_emo"]
    .value_counts(normalize=True)
    .rename("confidence")
    .reset_index()
)

label_agg = label_distribution.groupby("speaker_text").agg({
    "annotator_emo": "first",   
    "confidence": "max"       
}).reset_index()

label_agg.columns = ["speaker_text", "final_emotion", "label_confidence"]
label_agg.head(2)

,speaker_text,final_emotion,label_confidence
0,2gis брюс ли,neutral,0.5
1,a моей жене нет,other,0.5


In [ ]:
df_test = label_agg
print(df_test.shape)
df_test = df_test[df_test["speaker_text"].notna()]
print(df_test.shape)
df_test = df_test[df_test["speaker_text"].str.len() >= 5]
print(df_test.shape)

(16371, 3)
(16371, 3)
(16294, 3)


In [104]:
df_test["final_emotion"].value_counts(normalize=True)

final_emotion
neutral     0.632994
sad         0.142691
angry       0.122069
positive    0.089665
other       0.012581
Name: proportion, dtype: float64

In [ ]:
df_test = df_test[df_test["label_confidence"] >= 0.5]
print(df_test.shape)

(15800, 3)


In [ ]:
df_test["final_emotion"] = df_test["final_emotion"].replace("other", "neutral")

df_test["final_emotion"].value_counts(normalize=True)

final_emotion
neutral     0.656582
sad         0.136772
angry       0.118354
positive    0.088291
Name: proportion, dtype: float64

In [107]:
df_test.drop(columns = 'label_confidence', inplace = True)

In [108]:
df_test

,speaker_text,final_emotion
0,2gis брюс ли,neutral
1,a моей жене нет,neutral
2,a тупой что ли давай зеленоглазое такси,angry
3,afina да,neutral
4,afina т,neutral
...,...,...
16366,ярослав сумишевский концерт в ялте,neutral
16367,ярославская еще летом,neutral
16368,ясно дело ты просто тупой бот,angry
16369,яхт клуб балтиец,positive


In [109]:
df_test.to_csv('dusha_crowd_test_processed.csv')

In [110]:
df = pd.concat([df_train, df_test])
df.shape

(129953, 2)

In [111]:
df_test.shape[0] + df_train.shape[0]

129953

In [112]:
df.to_csv('dusha_crowd_processed.csv')

In [3]:
df = pd.read_csv('C://Users//a.karakuchukova//Desktop//Университет//Диплом//DataSets//dusha.crowd//dusha_crowd_processed.csv')

In [4]:
df.head(2)

,Unnamed: 0,speaker_text,final_emotion
0,0,2drots литвин,neutral
1,2,2gis брюс ли,positive


In [5]:
df.final_emotion.value_counts()

final_emotion
neutral     87574
sad         18070
positive    13101
angry       11208
Name: count, dtype: int64